In [ ]:
import os
from scipy import signal
import mne
import copy
import matplotlib.pyplot as plt
from math import sqrt
from mne import io
from mne.io import concatenate_raws, read_raw_edf
import numpy as np
# 数据可视化包
import matplotlib.pyplot as plt
import pandas as pd
from scipy.fft import fft
from scipy.stats import entropy

In [ ]:
#######这是所有的通道
all_the_channels = ['EEG Fp1-LE', 'EEG F3-LE', 'EEG C3-LE',
                                       'EEG P3-LE', 'EEG O1-LE', 'EEG F7-LE', 
                                       'EEG T3-LE', 'EEG T5-LE', 'EEG Fz-LE',
                                       'EEG Fp2-LE', 'EEG F4-LE','EEG C4-LE',
                                       'EEG P4-LE', 'EEG O2-LE', 'EEG F8-LE',
                                       'EEG T4-LE', 'EEG T6-LE', 'EEG Cz-LE',
                                       'EEG Pz-LE', 'EEG A2-A1', 'EEG 23A-23R',
                                       'EEG 24A-24R']
#######可以自己挑一些通道出来
picked_channels = ['EEG Fp2-LE', 'EEG F4-LE','EEG C4-LE',
                   'EEG P4-LE', 'EEG O2-LE', 'EEG F8-LE']
picked_channels = ['EEG Fp1-LE', 'EEG F3-LE', 'EEG C3-LE',
                                       'EEG P3-LE', 'EEG O1-LE', 'EEG F7-LE', 
                                       'EEG T3-LE', 'EEG T5-LE', 'EEG Fz-LE',
                                       'EEG Fp2-LE', 'EEG F4-LE','EEG C4-LE',
                                       'EEG P4-LE', 'EEG O2-LE', 'EEG F8-LE',
                                       'EEG T4-LE', 'EEG T6-LE', 'EEG Cz-LE',
                                       'EEG Pz-LE', 'EEG A2-A1', 'EEG 23A-23R',
                                       'EEG 24A-24R']


def extract_features_alpha(data,label,f1,f2,t1,t2):
   
   ###提取alpha非对称特征
   data_psd = data.compute_psd(method='welch',tmin=t1, tmax=t2, fmin=0.5, fmax=30, picks=None).get_data()
   alpha_psd = data.compute_psd(method='welch',tmin=t1, tmax=t2, fmin=f1, fmax=f2, picks=None).get_data()
   Slmn_f1_f2 = np.sum(alpha_psd, axis=1, keepdims=True)
   Slmn_f_all = np.sum(data_psd, axis=1, keepdims=True)
   Wmn = Slmn_f1_f2/Slmn_f_all

   alpha_feature_list = []
   for i in range(8):
       for j in range(8):
           F = (Wmn[i] - Wmn[j+9]) / (Wmn[i]+Wmn[j+9])
           F = float(F*100)
           alpha_feature_list.append(F)
   alpha_feature_list.append(label)
   alpha_feature_df = pd.DataFrame([alpha_feature_list])
   
   return alpha_feature_df
def extract_features_activity(data,label):
   data = data.to_data_frame()
   random_subset = data.sample(n=288)
   variances = random_subset.iloc[:, 1:].var()
# 创建包含特征名和方差结果的DataFrame
   result_df = pd.DataFrame({ 'Variance': variances.values})
# 转置DataFrame
   result_df = result_df.T
   result_df['label'] = label

   return result_df



def extract_features_FE(data,label):
   data = data.to_data_frame()
   columns = data.columns[1:]

# 初始化存储FE特征的DataFrame
   fe_features = pd.DataFrame()
   for column in columns:
    # 从数据中提取当前通道的数据
    channel_data = data[column].values

    # 对信号进行傅立叶变换
    fft_data = fft(channel_data)

    # 计算频率分布的概率密度函数（PDF）
    pdf, bins = np.histogram(np.abs(fft_data), bins=10, density=True)

    # 使用概率密度函数计算频率熵（FE特征）
    fe_value = entropy(pdf)

    # 将结果存储到fe_features中
    fe_features[column + '_fe'] = [fe_value]
   fe_features['label'] = label
   return fe_features

def extract_features_mobility(data,label):
   data = data.to_data_frame()
   columns = data.columns[1:]
   # 建立一个空数组来存储每一列的方差
   variances = []
   diff_variances = []
   mobilitys = pd.DataFrame()
   for column in columns:
    variance = data[column].var()
    diff = data[column].diff()
    diff_variance = diff.var()
    mobility=sqrt(diff_variance/variance)
    variances.append(variance)
    diff_variances.append(diff_variance)
    mobilitys[column]=[mobility]
   
   mobilitys = mobilitys
   mobilitys['label']=label
   return mobilitys

def extract_features_activity(data,label):
   data = data.to_data_frame()
   random_subset = data.sample(n=288)
   variances = random_subset.iloc[:, 1:].var()
# 创建包含特征名和方差结果的DataFrame
   result_df = pd.DataFrame({ 'Variance': variances.values})
# 转置DataFrame
   result_df = result_df.T
   result_df['label'] = label

   return result_df



def get_data_and_label(data_path,channels):
    edf_files = [file for file in os.listdir(data_path) if file.endswith(".edf")]
    data_list = []
    label_list = []

    for file in edf_files:
     file_path = os.path.join(data_path, file)
     data = read_raw_edf(file_path,preload=True,include=channels)
###########滤波
     
     if file.startswith("H"):
        label = 0
        
     elif file.startswith("MDD"):
        label = 1
        
     else:
        label = None

     if label is not None:
        #data_list.append(extract_features(data))
        data_list.append(data)
        label_list.append(label)
    return data_list,label_list
##从一个edf中提取出特征矩阵

data_list,label_list = get_data_and_label(data_path='./data',channels=picked_channels)
    

In [ ]:
####舍弃前30秒和后30秒
time_to_abandon = 30
###时间间隔和步长
T = 45
step = 1 ###step = T则代表不进行重叠



In [ ]:
####提取每 T 秒一段,步长为 step 的不对称性

all_the_features = pd.DataFrame()
for i in range(179):
    end_sample_time = int(data_list[i].n_times/265)
    for m in range(time_to_abandon,end_sample_time-time_to_abandon,step):
        one_feature = extract_features_alpha(data_list[i],label=label_list[i],f1=4,f2=7,t1=m,t2=m+T)
        all_the_features = pd.concat([all_the_features,one_feature],axis=0)
all_the_features.to_csv('./features/theta_{}_seconds_with_step_{}.csv'.format(T,step),index=False)   

In [ ]:
# ####提取移动性特征
# all_the_features = pd.DataFrame()
# for i in range(179): 
#         one_feature = extract_features_mobility(data_list[i],label=label_list[i])
#         all_the_features = pd.concat([all_the_features,one_feature],axis=0)
# all_the_features.to_csv('./features/the_mobility.csv',index=False)

In [ ]:
####提取活动性特征
# all_the_features = pd.DataFrame()
# for i in range(179): 
#         one_feature = extract_features_activity(data_list[i],label=label_list[i])
#         all_the_features = pd.concat([all_the_features,one_feature],axis=0)
# all_the_features.to_csv('./features/the_activity.csv',index=False)
    